# IN3050/4050 - Week 11: Reinforcement Learning

**Reinforcement Learning** (RL) is the last machine learning paradigm covered in this course. We will only take a cursory look at it. There is a lot of depth to this topic (just as for *supervised* and *unsupervised* learning), and we can of course not cover it in one single week. For those that are interested we can recomend the book "Reinforcement Learning: An Introduction" by Richard S. Sutton and Andrew G. Barto.

In these exercises we will focus on understanding the main concepts, and implementing basic reinforcement learning scenarios and solutions.

# Theory

Let us first redefine the main concepts underlying reinforcement learning:
1. What is an agent?
2. What is an environment?
3. What is a state?
4. What is an action?
5. What is a policy?
6. What is a reward?

**Answer**: 
1. **Agent**: An entity interacting with an enviornment according to a policy. 

2. **Enviroment**: The entity with wich the agent interact, and from which the agent receives feedbacks and reward. 

3. **State**: A descriptor of the environment and the agent (a sfficicent statistics of the history for teh agents decision). 

4. **Action**: A way for the agent to interact with the eenvionment. 

5. **Policy**: A rule to choose actions given states. 

6. **Reward**: A signal denoting how good the current action and, implicitily, the preceding ones where.  

# Exercise

In this section we are going to run some simulations on a simple *GridWorld* environment.

## Setup
We first import some support libraries.

In [3]:
import numpy as np

import constants as const
import policies as plcs
import worlds as wlds

The module **constants** simply contains some handy aliases. Actions and objects in the environment are identified by integer numbers. This module allows you to refer to actions using the format *const.LEFT* instead of an integer number; similarly objects in the world can be referred to *const.WALL* instead of an integer number. Check out the source file if you want to see the association between aliases and their integer encoding.

The module **policies** contains a set of pre-made policies that we will use in the exercise.

The module **worlds** contains a set of pre-made gridworlds that we will use in the exercise.

## Environment: GridWorld
Our environment is a simple GridWorld, that is a discrete flat rectangular environment made up $N\times M$ squares, thus defining a grid. One square is identified as the *starting position*, and another square is the *ending position*. An agent start in the starting position, and its aim is to reach the ending position. A gridworld may be enriched with additional *walls* (that is, squares where the agent can not pass) and *traps* (that is, squares that kill the agent and terminate the episode).

This gridworld model is **completely deterministic** (actions never fail) and **completely observed** (the agent perfectly knows the state of the world). 

In [4]:
import gridworld as gw

The module **gridworld** contains the class *GridWorld* implementing the environment with which the agent interact. An object *GridWorld* offers the following interface:
- **GridWorld(width,height, staring_position_x,staring_position_y, ending_position_x,ending_position_y)**: this is the constructor for a gridworld. It expects six parameters: the *width* and the *length* of the gridworld; the coordinates x and y of the starting position (*staring_position_x,staring_position_y*) and of the ending position (ending_position_x,ending_position_y). It returns an instance of a gridworld.
- **add_wall(x,y)**: this is a function to add a wall to the gridworld in location *x,y*.
- **add_trap(x,y)**: this is a function to add a trap to the gridworld in location *x,y*.
- **step(action)**: this is a function that takes an *action* passed as a parameter. The gridworld processes the action and returns four values: the new *state* of the world (that is, the new position of the agent in the form of a Loc object with two attributes x and y); the *reward* obtained by the agent by performing the action; a boolean *termination* indicating whether the episode terminated; and, optionally, some debug messages. 
- **reset()**: this is a function to reset the environment to the starting condition; it returns the same values as step().
- **print_basic()**: this is a function to produce a simple textual representation of the environment on the screen.
- **print_unicode()**: this a slightly fancier printing function using Unicode characters (if support is available).

**CAUTION**: A gridworld is indexed as a numpy matrix. Thus, if you build a $10 \times 10$ matrix, the height and the width of the gridworld will be 10; however the square in the world will be indexed from 0 to 9, with (0,0) being the upper-left corner square, and (9,9) being the lower-right corner square. 


An example $3 \times 3$ world:
![title](grid.png)

**Verification of consistency in *YOUR* hands!**

## Playing with the GridWorld

Create now a $4 \times 4$ gridworld with starting position in the upper-left corner and ending position in botttom-right.

In [5]:
W = gw.GridWorld(4,4,0,0,3,3)

Let us add a wall near the center in location (1,2).

In [6]:
W.add_wall(1,2)

Visualize the world using one of the supported print functions.

In [37]:
W.print_basic()
W.print_unicode()

[[  1    0    0    0 ]
 [  0    0    0    0 ]
 [  0    0    0    0 ]
 [  0    0    0    8 ]]


Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺




In the basic rendering 0 is an open space, 1 is the starting position, 6 is a trap, 7 is a wall, 8 is the agent and 9 is the ending position.

In the unicode rendering a white square is an open space, alpha is the starting position, a cross is a trap, a black square is a wall, the agent is a smile and omega is the ending position.

Resetting is a useful operation to restart the environment clean. It is a good practice to call it before starting any simulation to be certain that the environment is in the starting state. Call the reset function.

In [8]:
state, reward, termination, msgs = W.reset()

Take the action of a step to the right, print the return values of the action, visualize the world again, and comment on the action and its consequences. 

In [9]:
print(W.step(const.RIGHT))
W.print_unicode()

(<gridworld.Loc object at 0x1054327e0>, -1, False, ['I moved', None, None])
Α ☺ □ □
□ □ ■ □
□ □ □ □
□ □ □ Ω




**Comment**: Legit move, we get closer to the goal. 

Try now to take a step in the up direction, print the return values of the action, visualize, and comment on the action.

In [10]:
print(W.step(const.UP))
W.print_unicode()

(<gridworld.Loc object at 0x1054327e0>, -1, False, ['I can not move out of the world', None, None])
Α ☺ □ □
□ □ ■ □
□ □ □ □
□ □ □ Ω




**Comment**: We see that we cant move out of the world and get a penalty for that. 

Lead the agent to the objective, print the return values of the action, visualize, and comment on the actions and the rewards.

In [38]:
print(W.step(const.DOWN))
W.print_unicode()
print(W.step(const.DOWN))
W.print_unicode()
print(W.step(const.RIGHT))
W.print_unicode()
print(W.step(const.DOWN))
W.print_unicode()
print(W.step(const.RIGHT))
W.print_unicode()

(<gridworld.Loc object at 0x1054c4680>, -1, True, 'Game is over')
Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺


(<gridworld.Loc object at 0x1054c4680>, -1, True, 'Game is over')
Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺


(<gridworld.Loc object at 0x1054c4680>, -1, True, 'Game is over')
Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺


(<gridworld.Loc object at 0x1054c4680>, -1, True, 'Game is over')
Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺


(<gridworld.Loc object at 0x1054c4680>, -1, True, 'Game is over')
Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺




**Comment**: We see for each move we get -1 and when we reach the goal we get 100. 

Reset the world to the starting state, print the return values of the action, visualize, and comment on the result.

In [39]:
# Reset the world
state, reward, termination, msgs = W.reset()
W.print_unicode()

☺ □ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω




**Comment**: We are now at the start. 

## Policy Evaluation

One of the main problems in reinforcement learning is **policy evaluation**: *given a policy for acting in the environment, how good is the policy?*

We will consider and answer this question first for simple *deterministic* policies (computing the simple overall return of a policy) and then for *stochastic* policies (using Monte Carlo to estimate the average overall return of a policy).

### Setting up the environment
First load *world1()* from *wrlds* and visualize it.

In [13]:
W = wlds.world1()
W.print_unicode()

☺ □ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω




### Running a deterministic policy
We will first use the **deterministic policy** *plcs.policy1()*. This defines a policy that moves the agent always to the right until it reaches the border, and then heads down.

Design a loop that makes the agent interact with the environment using this policy until termination. Visualize the output at each step to verify that the policy works as expected.

In [14]:
state, reward, termination, msgs = W.reset() 

while not termination:
    action = plcs.policy1(state)
    state,reward, termination,msgs = W.step(action)
    W.print_unicode()

Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ ☺
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ ☺
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺




Edit the previous loop to track the states traversed, the actions taken, and the rewards collected.

In [15]:
states = []
actions = []
rewards = []

state,reward, termination,msgs = W.reset()
states.append(state)

while not termination:
    action = plcs.policy1(state)
    actions.append(action)
    state,reward, termination,msgs = W.step(action)
    states.append(state)
    rewards.append(reward)
    W.print_unicode()

Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ ☺
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ ☺
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺




### Computing the return of a deterministic policy
One of the main quantities to evaluate the goodness of a policy is the *return*. While a *reward* quantifies the local/short-term goodness of a single action, the *return* quantifies the global/long-term goodness of a policy.

Remember that we can compute the overall return of a policy $\pi$ from time $t$ using a discount $\gamma$ as:

$$
G_{\pi}(t) = r(t) + \sum_i \gamma^i r(t+i),
$$
where $G_{\pi}(t)$ is the return at time $t$ and r(t) is the reward at time $t$ under policy ${\pi}$. 

Write code to compute the return of policy *plcs.policy1()* from the initial state at $t=0$ with a discount rate of $\gamma = 0.9$.

In [16]:
gamma = 0.9

G = 0

for i in range(len(rewards)):
    G += rewards[i] * (gamma ** i)

print('Return: {0}'.format(G))

Return: 54.953900000000004


Run now the same code using the policy *plcs.policy2()*. This defines a deterministic policy that moves the agent first downwards, then upwards, and finally right and down to the objective. Plot the policy, compute its return and comment on the two policies you analyzed.

In [17]:
# Initialize empty lists for new trajectory
states = []
actions = []
rewards = []

# Reset the environment
state, reward, termination, msgs = W.reset()

# Add initial state
states.append(state)

# Run episode with policy2
while not termination:
    # Get action from policy2
    action = plcs.policy2(state)
    
    # Store the action
    actions.append(action)
    
    # Take the action
    state, reward, termination, msgs = W.step(action)
    
    # Store state and reward
    states.append(state)
    rewards.append(reward)
    
    # Visualize
    W.print_unicode()

# Compute return with γ = 0.9
gamma = 0.9
G = 0
for i in range(len(rewards)):
    G += rewards[i] * (gamma ** i)

print('Return: {0}'.format(G))

Α □ □ □
☺ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
☺ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
☺ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ ☺ □ Ω


Α □ □ □
□ □ □ □
□ ☺ □ □
□ □ □ Ω


Α □ □ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ ☺
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ ☺
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺


Return: 24.519165569900004


**Comment**: We see policy 1 is more direct since we end up with 54 compared to policy2 where we end up with 24. 

### Limitation of the deterministic policy
The deterministic policy *plcs.policy1()* is optimal for *wlds.world1()*. However deterministic policies have severe limitations.

Load *world2* from *wlds*, plot the environment, and run 10 steps of *plcs.policy1()*. Comment on the results.

In [18]:
W = wlds.world2()
W.print_unicode()

for i in range(10):
    action = plcs.policy1(state)
    state,reward, termination,msgs = W.step(action)
    W.print_unicode()

    if termination:
        break

☺ □ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ■ □
☺ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ■ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Α □ ■ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Α □ ■ □
□ □ □ ☺
□ □ □ □
□ □ □ Ω


Α □ ■ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Α □ ■ □
□ □ □ □
□ □ □ □
□ □ □ ☺




**Comment**: We see that we find the goal profecient. 

### Running a stochastic policy

Having assessed the limitation of a deterministic policy, we now consider a **stochastic policy**. A stochastic policy adds some randomness in the selection of the action to perform. Taking random actions will allow the agent to *explore* more of the world and will provide a higher degree of *generalization*/*adaptability*.
We use here *plcs.policy_epsilon1()*, a stochastic variant of *plcs.policy1()*, that behaves according to *plcs.policy1()*, except that 20% of the time it will take a random action.

Run *plcs.policy1()* both on *world1* and *world2* and comment on the results. (You may want to run your simulation more than one time).

In [19]:
W = wlds.world1()
W.print_unicode()

state, reward, termination, msgs = W.reset()
while not termination:
    action = plcs.policy_epsilon1(state)
    state,reward,termination,msgs = W.step(action)
    W.print_unicode()

☺ □ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ ☺
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ ☺
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺




In [20]:
W = wlds.world2()
W.print_unicode()

state, reward, termination, msgs = W.reset()
while not termination:
    action = plcs.policy_epsilon1(state)
    state,reward,termination,msgs = W.step(action)
    W.print_unicode()

☺ □ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ ■ □
□ □ □ 

**Comment**: We see that the world1 is and world2 is more random now and in world2 we are stuck longer before moving. 

### Computing the return of a stochastic policy
As in the case of deterministic policies, we want to be able to quantify the goodness of a stochastic policy. However, the simple approach we adopted above will not immediately work in the case of a stochastic policy. Any two simulations using a stochastic policy will likely return different results, and computing returns on each of them will yield different values.

### Monte Carlo Policy Evaluation (return)

As discussed above, we can not evaluate the return of a stochastic policy deterministically from a single simulation. Indeed, two different simulations would yield different returns. An intuitive solution to this problem would be to compute the return of the stochastic policy *in expectation*.

**Monte Carlo methods** constitute a wide family of techniques based on simulation and estimation of expected quantities. Imagine you are given a system whose dynamics are unknown or too hard to analyze analytically; suppose also that the system can be cheaply simulated; then we can try to answer questions about this system by repeatedly running simulations, collecting samples, and computing statistical quantities about the system.

In our case, the dynamics of the environment (how we transition from one state to another given an action) are known and we could theoretically compute analytically the value of the policy. However we assume that this computation is too expensive and we rely on a Monte Carlo approximation. 

What we (and the agent) do is trying out a given policy many times; each repetition, from the starting state to termination, is called an *episode*; the list of actions, states and rewards collected during an episode is called a *trajectory*. Given a set of trajectories, we can compute the expected value of our policy.

As you know, *Monte-Carlo* is just one approach to (tabular) policy evaluation in the case of stochastic policies. Alternatives are *dynamic programming* (where we assume the agent knows the dynamics of the environment) or *temporal differences* (where we assume that we do not have complete episodes).

Load *wlds.world1()* and *plcs.policy_epsilon1()*, and run 50 episodes with this setting while collecting the trajectories of your simulations.

In [21]:
n_episodes = 50
episodes = []

# Run 50 episodes
for i in range(n_episodes):
    # Initialize lists for this episode's trajectory
    states = []
    actions = []
    rewards = []
    
    # Reset environment
    state, reward, termination, msgs = W.reset()
    
    # Add initial state
    states.append(state)
    
    # Run episode until termination
    while not termination:
        # Get action from stochastic policy
        action = plcs.policy_epsilon1(state)
        
        # Store action
        actions.append(action)
        
        # Take action
        state, reward, termination, msgs = W.step(action)
        
        # Store state and reward (ensure reward is a number)
        states.append(state)
        rewards.append(float(reward))  # Convert reward to float when storing
    
    # Store this episode's trajectory
    episodes.append({
        'states': states,
        'actions': actions,
        'rewards': rewards  # Now contains only float values
    })

We can now use the data we have collected to compute the return in expectation:
$$ G_{\pi}(t) = \mathbb{E} \left[r(t) + \sum_i \gamma^i r(t+i)\right]. $$

This simply corresponds to the average return over the episodes:
$$ \hat{G}_{\pi}(t) = \frac{1}{N_e} \sum_{e=1}^{N_e} \left( r^{(e)}(t) + \sum_i \gamma^i r^{(e)}(t+i) \right), $$
where $r^{(e)}$ is the reward obtained during episode number $e$, and $N_e$ is the number of episodes collected.

Compute the expected return of policy *plcs.policy_epsilon1()* from the trajectories you collected, compare it with the return of the deterministic policy and comment.

In [22]:
gamma = 0.9  # Discount factor - future rewards are worth 90% of immediate rewards
totG = 0.0   # Initialize total return across all episodes

# Loop through each of the 50 episodes
for i in range(n_episodes):
    # Get the list of rewards from this episode's trajectory
    episode_rewards = episodes[i]['rewards']
    
    # Calculate the discounted return for this specific episode
    G = 0
    for t in range(len(episode_rewards)):
        # At each timestep t:
        # - episode_rewards[t] is the reward at that time
        # - gamma ** t is the discount (0.9^t)
        # - Multiply them and add to episode return
        G += episode_rewards[t] * (gamma ** t)
    
    # Add this episode's return to the running total
    totG += G

# Calculate the average return across all episodes
hatG = totG / n_episodes   
print('Return: {0}'.format(hatG))

Return: 10.147183657386634


**Comment**: enter your comment here.

## Policy improvement

So far, we have only considered the problem of evaluating how good a given policy is. We now consider a second central problem in reinforcement learning, that is, **policy improvement**: *given a policy, how can we construct a better policy?*

### State values and Action-state values
In the previous section we have evaluated policies by computing or estimating their overall return. This is a **global** value that assesses the goodness of a predefined policy from the beginning to the end. Comparing two policies according to their overall return is simple. But improving policies relying only on this quantity is tricky. Generating new policies, computing their overall return, and comparing with existing policies is unfeasible, as the number of possible policies explodes with the number of states and actions.

Ideally, we need some form of **local** information that would allow us to assemble new policies in a *modular* way. These quantities are:
- **state values** $v(s)$: evaluating how good is it to be in a certain state $s$ (more precisely, what is the expected return for a certain state $s$).
- **action-state values** $q(s,a)$: evaluating how good is it to take a certain action $a$ being in a certain state $s$ (more precisely, what is the expected return from a certain state $s$ by taking action $a$).
By knowing locally how good is a certain state or how good is a certain action we can steer the choices towards better policies.

### State values

The state value $v(s)$ of a state $s$ under the policy $\pi$ is defined as:
$$
v_{\pi}(s) = \mathbb{E} \left[ G(t) \vert S(t)=s \right],
$$
that is the return, as we computed above, but starting now from state $s$ instead of the initial state.

As before, computing the return for a stochastic policy is not immediate, and it requires relying on one of the approaches referenced above (*dynamic programming, Monte Carlo, temporal differences*).

### Monte Carlo Policy Evaluation for State Values

The same Monte Carlo principle we used to estimate overall returns may be used to compute state values: we run several episodes of the agents acting in the world and then we compute the state values.

First we generate the data. We use the same environment, but we now consider as a starting policy a completely random policy. This choice represents our complete ignorance about the environment (no one provided us with an almost optimal policy) and it will allow the agent to explore the environment more widely.

Load *wlds.world1()* and *plcs.policy_random()*, and run 100 episodes with this setting while collecting the trajectories of your simulations.

In [23]:
n_episodes = 100
episodes = []

# Load world1
W = wlds.world1()

# Run 100 episodes with random policy
for i in range(n_episodes):
    # Initialize lists for this episode's trajectory
    states = []
    actions = []
    rewards = []
    
    # Reset environment
    state, reward, termination, msgs = W.reset()
    
    # Add initial state
    states.append(state)
    
    # Run episode until termination
    while not termination:
        # Get action from random policy
        action = plcs.policy_random(state)
        
        # Store action
        actions.append(action)
        
        # Take action
        state, reward, termination, msgs = W.step(action)
        
        # Store state and reward (ensure reward is a float)
        states.append(state)
        rewards.append(float(reward))
    
    # Store this episode's trajectory
    episodes.append({
        'states': states,
        'actions': actions,
        'rewards': rewards
    })

We are going to compute the state value function $v(s)$ of each state $s$ using the *first-visit* Monte Carlo algorithm. For each state $s$, we find the first instance in an episode and compute its return. At the end we compute the expected return.

Compute the state value function using the first-visit Monte Carlo algorithm. 
1. Iterate over every state; a state is encoded as *gw.Loc()* object from the module *gw*; the constructor *gw.Loc(x,y)* receives the $x,y$ coordinates of a square in the grid world.
2. Iterate over the episodes; in each episode find the first occurence of the current state; you can use the function *gw.find_location_in_array()* from the module *gw*. The function *gw.find_location_in_array(loc, list)* receives a location *loc* in format *gw.Loc()* and a list of locations; it returns the index of the first instance of *loc* within *list*, or *None* if *loc* does not appear in *list*.
3. Compute the return from the first occurrence of the current state.
4. Average the return over all the episodes.

In [24]:
Vs = {}  # Dictionary to store state values
gamma = 0.9  # Discount factor

# Iterate over all possible states in 4x4 grid
for x in range(4):
    for y in range(4):
        s = gw.Loc(x,y)  # Create state location
        Gs = []  # List to store returns for this state
        
        # Iterate over all episodes
        for episode in episodes:
            # Find first occurrence of state s in this episode
            first_visit_idx = gw.find_location_in_array(s, episode['states'])
            
            # If state was visited in this episode
            if first_visit_idx is not None:
                # Calculate return from first visit onwards
                G = 0
                rewards = episode['rewards'][first_visit_idx:]  # Get rewards from first visit
                
                # Calculate discounted return
                for t in range(len(rewards)):
                    G += rewards[t] * (gamma ** t)
                
                # Add this return to list of returns for state s
                Gs.append(G)
        
        # Average returns for this state if we have any visits
        if Gs:
            Vs[s] = np.mean(Gs)
        else:
            Vs[s] = 0  # If state never visited, set value to 0

Now place the state-values in a matrix V (respecting the convention discussed at the beginning on the labelling of the squares), print the matrix, and comment on the result.

In [25]:
V = np.zeros((4,4))
for x in range(4):
    for y in range (4):
        state = gw.Loc(x,y)
        V[x,y] = Vs[state] if state in Vs else 0
        
print(V)

[[-1.01244966e+00  3.35724787e-03  2.29575557e+00  6.92653852e+00]
 [ 1.19151356e+00  4.08934963e+00  1.06043939e+01  1.90127456e+01]
 [ 2.12600005e+00  8.44517031e+00  2.47479133e+01  4.01773654e+01]
 [ 4.11599180e+00  1.42951578e+01  4.01484889e+01  0.00000000e+00]]


**Comment**: The matrix shows a gradient toward the terminal state. 

### Policy Improvement
Our agent started with a purely random policy. Now that we have computed the state values of our grid world we can select an optimal policy. A simple way to use the state-value matrix to improve on our random policy is the following one: at each state select the action that leads you to the successor state with highest value.

Define a policy (a function receiving a state location as an input) using the state-value matrix you computed above to define an improved policy.

In [26]:
def my_policy(state):
    value_next_state = {}
    
    if(state.x>0):
        value_next_state[const.UP] = V[state.x-1,state.y]
    else:
        value_next_state[const.UP] = -np.inf
    # Check DONW move if not at bottom edge
    if(state.x < 3):
        value_next_state[const.DOWN] = V[state.x+1,state.y]
    else:
        value_next_state[const.DOWN] = -np.inf

    #Check LEFT move if not at left edge
    if(state.y >0):
        value_next_state[const.LEFT] = V[state.x, state.y-1]
    else:
        value_next_state[const.LEFT] = -np.inf

    # Check RIGHT move if not at right edge
    if(state.y < 3):
        value_next_state[const.RIGHT] = V[state.x,state.y+1]
    else:
        value_next_state[const.RIGHT] = -np.inf
        
    return max(value_next_state,key=value_next_state.get)

This sort of policy is called a *greedy policy* because, at each step, it always deterministically takes the action that is expected to produce the higher return.

Load *wlds.world1()* and use the greedy policy you have just defined to control the agent. Plot the outcome, and comment.

In [27]:
# Load world1
W = wlds.world1()

# Reset environment
state, reward, termination, msgs = W.reset()

max_steps = 100
step_count = 0

# Run episode with our greedy policy
while not termination and step_count < max_steps:
    # Get action from our greedy policy
    action = my_policy(state)
    print(f"Step {step_count}: At position ({state.x}, {state.y}), taking action {action}")
    # Take action
    state, reward, termination, msgs = W.step(action)
    
    # Visualize current state
    W.print_unicode()
    step_count += 1

if step_count >= max_steps:
    print("Episode terminated due to maximum steps reached")
else:
    print("Episode terminated successfully")

Step 0: At position (0, 0), taking action 2
Α □ □ □
☺ □ □ □
□ □ □ □
□ □ □ Ω


Step 1: At position (1, 0), taking action 3
Α □ □ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Step 2: At position (1, 1), taking action 3
Α □ □ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Step 3: At position (1, 2), taking action 2
Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Step 4: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Step 5: At position (2, 3), taking action 0
Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Step 6: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Step 7: At position (2, 3), taking action 0
Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Step 8: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Step 9: At position (2, 3), taking action 0
Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Step 10: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Step 11: At position (2, 3), taking action 0
Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Step 12: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□

**Comment**: It uses a lot of time to find the goal. 

A completely deterministic policy as the one we defined is always susceptible to underperform if the environment undergoes any change.

Design a world, possibly with minimal changes from *wlds.world1()*, where the optimal policy just learned fails. Run this world for 10 iterations using the learned greedy policy. 

In [28]:
W = gw.GridWorld(4,4, 0,0, 3,3)
W.add_wall(2,3)

state, reward, termination, msgs = W.reset()
for _ in range(10):
    action = my_policy(state)
    print(f"Step {i}: At position ({state.x}, {state.y}), taking action {action}")
    state,reward,termination,msgs = W.step(action)

    W.print_unicode()

    if termination:
        print("Episode terminated")
        break

if i == 9:
    print("Maximum steps reached without finding goal")

Step 99: At position (0, 0), taking action 2
Α □ □ □
☺ □ □ □
□ □ □ ■
□ □ □ Ω


Step 99: At position (1, 0), taking action 3
Α □ □ □
□ ☺ □ □
□ □ □ ■
□ □ □ Ω


Step 99: At position (1, 1), taking action 3
Α □ □ □
□ □ ☺ □
□ □ □ ■
□ □ □ Ω


Step 99: At position (1, 2), taking action 2
Α □ □ □
□ □ □ □
□ □ ☺ ■
□ □ □ Ω


Step 99: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ ☺ ■
□ □ □ Ω


Step 99: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ ☺ ■
□ □ □ Ω


Step 99: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ ☺ ■
□ □ □ Ω


Step 99: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ ☺ ■
□ □ □ Ω


Step 99: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ ☺ ■
□ □ □ Ω


Step 99: At position (2, 2), taking action 3
Α □ □ □
□ □ □ □
□ □ ☺ ■
□ □ □ Ω




In general, while learning and acting in the environment, it is a good idea to preserve some degree of *stochasticity* to keep analyzing the world and improve your policy.

Improving the policy of an agent is a central problem in reinforcement learning. The problem of **control** consists in finding or approximating an optimal policy. A common approach, which we have followed in this notebook, consists in alternating a step of *policy evaluation* and a step of *policy improvement*. In the example above, we have started with a random policy and we have computed the associated state-value matrix (policy evaluation); then, we have defined a greedy policy using the state-value matrix (policy improvement). This generic learning approach is called **Generalized Policy Iteration**. On more complex problem, several iterations of evaluation-improvement may be required to approximate an optimal policy.

# Optional: Q-Learning

**Q-Learning** is an *off-policy*, *temporal difference* reinforcement learning algorithm. 
Off-policy means that the agent learns an optimal target policy $\pi_t$ while behaving according to a different behaviour policy $\pi_b$; this allows, for instance, the agent to behave according to a sub-optimal exploratory policy while learning and memorizing an optimal policy. Temporal difference means that the agent uses an algorithm that allows learning in real-time without the need to collect whole episodes (as it was in the case of Monte Carlo).

Q-Learning does not rely on any model of the environment, and as such, it works estimating *state-action values* $q(s,a)$ instead of *state values* $v(s)$.

First, load *wlds.world1()* and randomly initialize a matrix/tensor to store the state-action values. 

In [29]:
W = wlds.world1()
Q = np.random.uniform(low=0, high=0.1, size=(4, 4, 4))

While acting in the world, we are going to define as a *behaviour policy* an $\epsilon$-greedy policy that, for each state, takes a random action with probability $\epsilon$ or the optimal action defined by the state-action matrix $q(s,a)$ with probability $1-\epsilon$.

Implement this policy as a function receiving as input the current state in the form of *gw.Loc()*, the current state-action matrix $q(s,a)$, and the parameter $\epsilon$.

In [30]:
def behaviour_policy(state,Q,eps):
    if np.random.random() < eps:
        return np.random.choice([const.UP,const.DOWN,const.LEFT,const.RIGHT])
    else:
        q_values = Q[state.x, state.y, :]
        return np.argmax(q_values)

We start our learning with the following parameters: discount factor $\gamma = 0.9$, learning rate $\alpha=0.1$, exploration stochasticity $\epsilon = 0.4$.

In [31]:
alpha = 0.1
gamma = 0.9
epsilon = 0.4

We now implement and run Q-learning for one episode.

1. Given that we are in state $s$, we choose an action $a$ according to our behaviour policy *behaviour_policy()*
2. We take action $a$, receive reward $r$ and end up in state $s'$
3. We update the action-value function $q(s,a)$ according to the following formula:
$$
q(s,a) \leftarrow q(s,a) + \alpha \left[ r + \gamma \max_{a'} q(s',a') - q(s,a) \right]
$$
where $\max_{a'} q(s',a')$ is the action-value of the action $a'$ that maximizes the action-value for state $s'$. 

In [32]:
# Reset environment
state, reward, termination, msgs = W.reset()

# Run one episode
while not termination:
    # 1. Choose action using behavior policy (ε-greedy)
    action = behaviour_policy(state, Q, epsilon)
    
    # 2. Take action, get new state and reward
    next_state, reward, termination, msgs = W.step(action)
    
    # 3. Q-learning update
    # Current Q-value
    current_q = Q[state.x, state.y, action]
    
    # Max Q-value for next state
    next_max_q = np.max(Q[next_state.x, next_state.y, :]) if not termination else 0
    
    # Q-learning update formula
    Q[state.x, state.y, action] = current_q + alpha * (reward + gamma * next_max_q - current_q)
    
    # Move to next state
    state = next_state
    
    # Visualize current state
    W.print_unicode()

Α □ □ □
☺ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
☺ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
☺ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ ☺ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
☺ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
☺ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ ☺ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
☺ □ □ Ω


Α □ □ □
□ □ □ □
☺ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ ☺ □ □
□ □ □ Ω


Α □ □ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ ☺
□ □ □ □
□ □ □ Ω


Α □ □ ☺
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ ☺ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺




Observe the values in the action-value matrix, for instance in the pre-final states (2,3) or (3,2). Comment on the result.

In [33]:
# Print Q-values for state (2,3)
print("Q-values for state (2,3):")
print("UP:", Q[2,3,0])    # UP action
print("DOWN:", Q[2,3,1])  # DOWN action
print("LEFT:", Q[2,3,2])  # LEFT action
print("RIGHT:", Q[2,3,3]) # RIGHT action

print("\nQ-values for state (3,2):")
print("UP:", Q[3,2,0])    # UP action
print("DOWN:", Q[3,2,1])  # DOWN action
print("LEFT:", Q[3,2,2])  # LEFT action
print("RIGHT:", Q[3,2,3]) # RIGHT action

Q-values for state (2,3):
UP: 0.09316324640008916
DOWN: 0.08319644062058529
LEFT: 10.088350297937565
RIGHT: 0.060055683335476895

Q-values for state (3,2):
UP: 0.01981638408356259
DOWN: 0.03659605625347511
LEFT: 0.07246293084005005
RIGHT: 0.09050556937512302


**Comment**: The action leading to the final state has been correctly updated in Q. 

Use the *mean* and *max* operator over the actions to compute an average and a max version of a state-value matrix from the action-state matrix (i.e.: average/max over the action). Comment on the result.

In [34]:
V_mean = np.mean(Q,axis=2)
V_max = np.max(Q,axis=2)

print("Mean State Values (averaging over actions):")
print(V_mean)
print("\nMax State Values (maximum over actions):")
print(V_max)

Mean State Values (averaging over actions):
[[ 1.30479029e-03  3.62738352e-02 -1.12312094e-02  9.25242773e-04]
 [ 7.01739929e-03  5.96090276e-04  2.55581571e-02  4.81290890e-02]
 [ 1.61031730e-02  1.40143767e-03  1.20584767e-02  2.58119142e+00]
 [-5.68793300e-02 -2.67220641e-02  5.48452351e-02  5.75304335e-02]]

Max State Values (maximum over actions):
[[ 0.04543105  0.07344787  0.06429783  0.04137334]
 [ 0.03121148  0.02593439  0.05854883  0.09185315]
 [ 0.0742916   0.04372309  0.03413102 10.0883503 ]
 [ 0.02072496  0.02748921  0.09050557  0.07180548]]


**Comment**: enter your comment here.

Run Q-learning for $10$ more episodes (without resetting your action-value matrix!)

In [35]:
n_episodes = 10

for _ in range(n_episodes):
    state, reward, termination , msgs = W.reset()
    while not termination:
        action = behaviour_policy(state,Q,epsilon)
        next_state,reward, termination, msgs = W.step(action)

        current_q = Q[state.x, state.y, action]
        next_max_q = np.max(Q[next_state.x, next_state.y,:]) if not termination else 0
        Q[state.x, state.y,action] = current_q + alpha * (reward + gamma * next_max_q - current_q)
        # Update state
        state = next_state
        W.print_unicode()

Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


☺ □ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


☺ □ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


☺ □ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Α ☺ □ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ ☺
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ ☺
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ ☺ □
□ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Α □ □ □
□ □ ☺ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ ☺ □
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ ☺
□ □ □ Ω


Α □ □ □
□ □ □ □
□ □ □ □
□ □ □ ☺


Α □ □ □
☺ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
□ ☺ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
☺ □ □ □
□ □ □ □
□ □ □ Ω


Α □ □ □
☺ □ □ 

Print again from the action-state matrix, and comment on the results.

In [36]:
# Print Q-values for state (2,3)
print("Q-values for state (2,3):")
print("UP:", Q[2,3,0])    # UP action
print("DOWN:", Q[2,3,1])  # DOWN action
print("LEFT:", Q[2,3,2])  # LEFT action
print("RIGHT:", Q[2,3,3]) # RIGHT action

print("\nQ-values for state (3,2):")
print("UP:", Q[3,2,0])    # UP action
print("DOWN:", Q[3,2,1])  # DOWN action
print("LEFT:", Q[3,2,2])  # LEFT action
print("RIGHT:", Q[3,2,3]) # RIGHT action

Q-values for state (2,3):
UP: 1.1018280251295596
DOWN: 1.525047396746606
LEFT: 68.64974623506725
RIGHT: 4.504982503269069

Q-values for state (3,2):
UP: 0.01981638408356259
DOWN: 0.03659605625347511
LEFT: 0.07246293084005005
RIGHT: 0.09050556937512302


**Comment**: We now notice more clearly a gradient towards the final state. 

Consider how learning would change if you were to change the learning parameter, or if the environment were to change.

**Answer**: Alpha may change the speed of convergence, epsilon may change the degree of exloration. Even if the envoroment where to change, the agent would keep exploring it following its behaviour policy. 

## Function Approximators
So far we have solved the reinforcement learning problem using tables and matrices to store state-value, $v$, functions and action-value, $q$, functions. This is called the **tabular** approach to RL. This solution is simple and interpretable, but it has a severe limitation.

What happens if our gridworld becomes much bigger, extending to a 10000 by 10000 grid? How much memory would we need to store the state-value or the action-value function?

**Answer**: A lot of memory. 1000^2 * 8bytes = 0.8gb. For the action value function we need 4 times more 0.8*4gb=3.2gb. 

Can you imagine a way to solve this issue?

**Answer:** We could use a neural network. 

# Conclusions

In this notebook we studied the basics of **reinforcement learning** through the analysis on a simple *gridworld* environment of two core RL problems: **policy evaluation** and **policy improvement**.

We first considered the problem of policy evaluation, that is, the problem of assessing how good a policy is. For **deterministic policies** we solved the problem simply by computing the *total discounted return* of a policy. After realizing the limitation of deterministic policies, we turned to **stochastic policies**, and we computed their *return in expectation* relying on *Monte Carlo methods*.

Next we considered the problem of policy improvement, that is, the problem of learning optimal policies. For this task, we realized that a global value for a policy is not handy when improving a policy, and, instead local values would be more useful. We then computed **state values** and **action values** to assess a policy step by step, and used this values to take improved actions. This iterative approach, based on evaluating a policy and improving the policy, is an instance of the more generic method called **Generalized Policy Iteration**.

In the final optional part, we considered a more advanced RL algorithm, **Q-learning**. Q-learning allowed us to learn an optimal policy while interacting with an environment; this algorithm had two important features: (i) it learns in a continuous way, step-by-step, and not on complete episodes; (ii) it learns an optimal policy while behaving according to a different, more exploration-prone, policy.